In [1]:
import os
import pandas as pd
import json


In [2]:
# 读取字典
with open('dict/touse.json', 'r') as f:
    error = json.load(f)
    error=error['传送模设计科']

path = '/home/keae/Mould/data'
filelist = os.listdir(path)
filelist.sort()

# 识别错误
for file in filelist[1:4]:
    df = pd.read_excel(os.path.join(path, file),
                       header=1, sheet_name='传送模设计科')
    df = df[['异常单编码', '品管异常描述']]
    df = df.replace('\n', '', regex=True)
    df.insert(df.shape[1], '错误位置', '')
    df.insert(df.shape[1], '错误类型', '')
    df.insert(df.shape[1], '原文错误', '')

    for index, row in df.iterrows():
        des = row['品管异常描述']

        for s in error['错误位置']:
            if des.find(s) != -1:
                df.loc[index, '错误位置'] = df.loc[index, '错误位置']+s+','
        df.loc[index, '错误位置'] = df.loc[index, '错误位置'].strip(',')

        for k, v in error['错误类型'].items():
            if des.find(k) != -1:
                df.loc[index, '错误类型'] = df.loc[index, '错误类型']+v+','
                df.loc[index, '原文错误'] = df.loc[index, '原文错误']+k+','
        df.loc[index, '错误类型'] = df.loc[index, '错误类型'].strip(',')
        df.loc[index, '原文错误'] = df.loc[index, '原文错误'].strip(',')

    # print(df)
    print(os.path.join(path, file))

    df.to_excel('results/'+file+'.xls')

    # break


/home/keae/Mould/data/9.11-9.17.xls
/home/keae/Mould/data/9.18-9.24.xls


/tmp/ipykernel_5322/410487909.py:36: FutureWarning: As the xlwt package is no longer maintained, the xlwt engine will be removed in a future version of pandas. This is the only engine in pandas that supports writing in the xls format. Install openpyxl and write to an xlsx file instead. You can set the option io.excel.xls.writer to 'xlwt' to silence this warning. While this option is deprecated and will also raise a warning, it can be globally set and the warning suppressed.
  df.to_excel('results/'+file+'.xls')


/home/keae/Mould/data/9.25-10.8.xls


In [36]:
errors = {}

path = '/home/keae/Mould/data'
filelist = os.listdir(path)
filelist.sort()

# 根据标注生成字典
error = {'错误位置': [], '错误类型': {}}
for file in filelist[1:3]:
    print(os.path.join(path, file))

    df = pd.read_excel(os.path.join(path, file), header=1, sheet_name='传送模设计科')
    df = df[['异常单编码', '品管异常描述', '错误位置1', '错误位置2', '原错误类型', '归纳错误类型']]
    df = df.replace('\n', '', regex=True)

    for index, row in df.iterrows():
        if not pd.isna(row['错误位置1']) and row['错误位置1'] not in error['错误位置']:
            error['错误位置'].append(row['错误位置1'])
        if not pd.isna(row['错误位置2']) and row['错误位置2'] not in error['错误位置']:
            error['错误位置'].append(row['错误位置2'])

        if not pd.isna(row['归纳错误类型']):
            if row['归纳错误类型'] not in error['错误类型']:
                error['错误类型'][row['归纳错误类型']] = []

            if row['原错误类型'] not in error['错误类型'][row['归纳错误类型']]:
                error['错误类型'][row['归纳错误类型']].append(row['原错误类型'])

        # break

    # break

errors['传送模设计科'] = error
with open('dict/errors.json', 'w') as f:
    json.dump(errors, f, sort_keys=True, indent=4, ensure_ascii=False)

# print(errors)


/home/keae/Mould/data/9.11-9.17.xls
/home/keae/Mould/data/9.18-9.24.xls


In [37]:
from copy import deepcopy
with open('dict/errors.json', 'r') as f:
    errors = json.load(f)
with open('dict/expert.json', 'r') as f:
    expert = json.load(f)
errors = errors['传送模设计科']
expert = expert['传送模设计科']

# 将生成字典和人工字典合并
for i in expert['错误位置']:
    if i not in errors['错误位置']:
        errors['错误位置'].append(i)

for k, v in expert['错误类型'].items():
    errors['错误类型'].setdefault(k, []).extend(v)

touse = deepcopy(errors)
touse['错误类型'] = {}
for k, v in errors['错误类型'].items():
    for e in v:
        touse['错误类型'][e] = k

touse = {'传送模设计科': touse}
with open('dict/touse.json', 'w') as f:
    json.dump(touse, f, sort_keys=True, indent=4, ensure_ascii=False)
